# Telco Churn EDA

In [2]:
import pandas as pd
from huggingface_hub import hf_hub_download

In [3]:
repo_id = "Carson-Shively/telco-churn"
filename = "data/bronze/churn_history.parquet"

local_path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        repo_type="dataset",
        revision="main",
    )
print(local_path)

df = pd.read_parquet(local_path)
df.head()

/home/carsonshively24/.cache/huggingface/hub/datasets--Carson-Shively--telco-churn/snapshots/efd725a7e2b407f652911f0003f89ae81a64bcbd/data/bronze/churn_history.parquet


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5696-EXCYS,Male,0,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),19.45,369.05,No
1,7562-UXTPG,Female,0,No,No,13,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,70.15,886.7,No
2,2073-QBVBI,Female,0,Yes,No,41,No,No phone service,DSL,No,...,No,Yes,No,No,One year,No,Mailed check,35.45,1391.65,No
3,5060-TQUQN,Male,0,Yes,Yes,30,Yes,Yes,Fiber optic,No,...,No,Yes,No,No,Month-to-month,Yes,Bank transfer (automatic),83.55,2570.2,No
4,4350-ZTLPI,Female,0,Yes,No,53,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),108.95,5718.2,No


## Schema Check

In [12]:
df.columns.tolist()

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

**Notes**
- customerID is the entity identifier.
- Churn is the target label.
- All other columns are raw input features and will be processed into gold features downstream.


In [13]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

**Notes**
- TotalCharges should be numeric but is currently object -> coerce to numeric in the silver layer.
- Several binary categorical fields (Yes/No) are stored as object -> will require encoding downstream.


## Missingness

In [10]:
df.isna().mean().sort_values(ascending=False)

customerID          0.0
gender              0.0
SeniorCitizen       0.0
Partner             0.0
Dependents          0.0
tenure              0.0
PhoneService        0.0
MultipleLines       0.0
InternetService     0.0
OnlineSecurity      0.0
OnlineBackup        0.0
DeviceProtection    0.0
TechSupport         0.0
StreamingTV         0.0
StreamingMovies     0.0
Contract            0.0
PaperlessBilling    0.0
PaymentMethod       0.0
MonthlyCharges      0.0
TotalCharges        0.0
Churn               0.0
dtype: float64

## Value Validity

In [11]:
df.describe(percentiles=[0.01, 0.05, 0.95, 0.99])

,SeniorCitizen,tenure,MonthlyCharges
count,6339.000000,6339.000000,6339.000000
mean,0.161697,32.359836,64.693319
std,0.368202,24.535393,30.138598
min,0.000000,0.000000,18.250000
1%,0.000000,1.000000,19.200000
5%,0.000000,1.000000,19.650000
50%,0.000000,29.000000,70.300000
95%,1.000000,72.000000,107.355000
99%,1.000000,72.000000,114.900000
max,1.000000,72.000000,118.750000


**Notes**
- Numeric feature ranges look valid (no negative values, plausible bounds).
- SeniorCitizen is binary (0/1).
- tenure appears to be in months (0–72).
- MonthlyCharges values are within expected range.


## Duplicate Check

In [12]:
key = "customerID"
df.duplicated(subset=[key]).sum()
df[key].isna().sum()

np.int64(0)

## Categorical Stability

In [13]:
cat_cols = df.select_dtypes(exclude=["number"]).columns
df[cat_cols].nunique().sort_values(ascending=False)

customerID          6339
TotalCharges        5918
PaymentMethod          4
Contract               3
MultipleLines          3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
OnlineSecurity         3
InternetService        3
StreamingTV            3
StreamingMovies        3
gender                 2
PhoneService           2
Dependents             2
Partner                2
PaperlessBilling       2
Churn                  2
dtype: int64

**Notes**
- customerID is high-cardinality and serves only as the entity key → exclude from FeatureSpec.
- All other categorical features have low, bounded cardinality (<=4 values).
- No high-cardinality categoricals detected → standard encoding strategies are safe.
- Churn is the binary target label.


## Target Sanity

In [14]:
label = "Churn"
df[label].isna().mean()
df[label].value_counts(normalize=True)


Churn
No     0.735763
Yes    0.264237
Name: proportion, dtype: float64

**Notes**
- Target label shows moderate class imbalance (~74% No, ~26% Yes).
- Imbalance should be considered during model training and evaluation.


## Conclusions / Decisions
- Entity key: customerID (no duplicates expected).
- Target: Churn (binary), moderate class imbalance (~74/26).
- Type fix: TotalCharges is object -> coerce to numeric in silver.
- Categoricals: low cardinality (<=4) -> safe to encode.
- FeatureSpec will exclude customerID and Churn; include remaining features with fixed order.
